## Required Libraries

In [ ]:
import pandas as pd
import numpy as np


## Data Preprocessing

In [ ]:
# Read only the first two columns
#df = pd.read_csv("Waste_3_6.csv", usecols=[0, 1]) ### two months data

df = pd.read_csv("Waste_3_6_July_Nov.csv", usecols=[0, 1])  ### new data set

# Rename columns
df.columns = ["Datetime", "Flow[cfs]"]

# Convert to datetime (auto-detects all formats)
df["Datetime"] = pd.to_datetime(df["Datetime"], errors="coerce")

# Convert flow values to numeric
df["Flow[cfs]"] = pd.to_numeric(df["Flow[cfs]"], errors="coerce")

# Drop invalid rows and set datetime index
df = df.dropna(subset=["Datetime"]).set_index("Datetime")

# Count how many have missing flow values
missing_points = df["Flow[cfs]"].isna().sum()

# Display first few rows
print(df.head())
print("Missing point",missing_points)


### Resample into 10 min flow 

In [ ]:
# Resample to a strict 10-minute grid and compute the mean per bin
flow_10min = df.resample("10min").mean()

### detecting and handliing outliers if any

In [ ]:
# Compute quartiles
Q1 = flow_10min["Flow[cfs]"].quantile(0.25)
Q3 = flow_10min["Flow[cfs]"].quantile(0.75)
IQR = Q3 - Q1

# Bounds
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Identify outliers
outliers = flow_10min[
    (flow_10min["Flow[cfs]"] < lower_bound) | 
    (flow_10min["Flow[cfs]"] > upper_bound)
]

print(f"Total outliers detected: {len(outliers)}")
print(f"Lower bound: {lower_bound:.3f}, Upper bound: {upper_bound:.3f}")

In [ ]:
flow_10min.loc[
    (flow_10min["Flow[cfs]"] < lower_bound) | 
    (flow_10min["Flow[cfs]"] > upper_bound), 
    "Flow[cfs]"
] = None

### Interpolate missing and outlier gaps (time-based)

In [ ]:
# fill gaps smoothly if there are any using time-based interpolation
flow_10min = flow_10min.interpolate(method="time")

# Display first few rows
print(flow_10min.head())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Since I don't have access to your specific flow_10min DataFrame,
# I'll assume it contains time series data with a datetime index
# and one or more columns of flow measurements

# If your DataFrame is already defined, you can skip this part
# This is just an example of what the DataFrame might look like
# flow_10min = pd.DataFrame(your_data_here)

# Basic plotting of the entire DataFrame
fig, ax = plt.subplots(figsize=(12, 6))

# If flow_10min has multiple columns, this will plot all of them
flow_10min.plot(ax=ax)

# If you want to plot just one specific column, uncomment and modify this line:
# flow_10min['your_column_name'].plot(ax=ax)

# Enhance the plot with labels and title
plt.title('Flow Data (10-minute intervals)', fontsize=14)
plt.xlabel('Time', fontsize=12)
plt.ylabel('Flow Rate', fontsize=12)
plt.grid(True, alpha=0.3)

# Add a legend if there are multiple columns
plt.legend(loc='best')

# Rotate x-axis labels if they're overlapping
plt.xticks(rotation=45)

# Adjust layout to prevent cut-off labels
plt.tight_layout()

# Display the plot
plt.show()

# ML Models

In [ ]:
import numpy as np, pandas as pd, random
from math import sqrt
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
import joblib

In [ ]:
SEED=42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")


### Time-ordered split 70:15:15 

In [ ]:
N=len(flow_10min)
n_train=int(0.70*N); n_val=int(0.15*N); n_test=N-n_train-n_val
train_df=flow_10min.iloc[:n_train].copy()
val_df  =flow_10min.iloc[n_train:n_train+n_val].copy()
test_df =flow_10min.iloc[n_train+n_val:].copy()

In [ ]:
test_df 

 ### Scale on TRAIN only 

In [ ]:
scaler=MinMaxScaler()
scaler.fit(train_df[['Flow[cfs]']].values)
train_scaled=scaler.transform(train_df[['Flow[cfs]']].values)
val_scaled  =scaler.transform(val_df[['Flow[cfs]']].values)
test_scaled =scaler.transform(test_df[['Flow[cfs]']].values)

### Windowing (Sliding)

In [ ]:
H=36   # 6 hours ahead (10-min steps)
L=144  # 24 hours lookback (10-min steps)


In [ ]:
def make_windows(series_2d, lookback=L, horizon=H):
    x_list, y_list=[], []
    T=len(series_2d)
    end = T - lookback - horizon + 1
    for i in range(end):
        x = series_2d[i:i+lookback]                      # [L,1]
        y = series_2d[i+lookback:i+lookback+horizon,0]   # [H]
        x_list.append(x); y_list.append(y)
    if not x_list:
        return np.empty((0,lookback,1)), np.empty((0,horizon))
    return np.stack(x_list), np.stack(y_list)

X_train,y_train=make_windows(train_scaled,L,H)
X_val,y_val    =make_windows(val_scaled,L,H)
X_test,y_test  =make_windows(test_scaled,L,H)

class SeqDS(Dataset):
    def __init__(self,X,y): self.X=torch.from_numpy(X).float(); self.y=torch.from_numpy(y).float()
    def __len__(self): return len(self.X)
    def __getitem__(self,i): return self.X[i], self.y[i]

bs=128
train_loader=DataLoader(SeqDS(X_train,y_train),batch_size=bs,shuffle=True)
val_loader  =DataLoader(SeqDS(X_val,y_val),batch_size=bs,shuffle=False)
test_loader =DataLoader(SeqDS(X_test,y_test),batch_size=bs,shuffle=False)

### Model

In [ ]:
class LSTMForecaster(nn.Module):
    def __init__(self,input_dim=1,hidden=128,layers=2,dropout=0.2,horizon=H):
        super().__init__()
        self.lstm=nn.LSTM(input_dim,hidden,num_layers=layers,
                          dropout=dropout if layers>1 else 0.0,batch_first=True)
        self.head=nn.Sequential(nn.Linear(hidden,64), nn.ReLU(), nn.Linear(64,horizon))
    def forward(self,x):
        out,_=self.lstm(x)           # [B,L,hidden]
        last=out[:,-1,:]             # [B,hidden]
        return self.head(last)       # [B,H]

model=LSTMForecaster(horizon=H).to(device)


### Train

In [ ]:
crit=nn.MSELoss()
opt=torch.optim.Adam(model.parameters(),lr=1e-3)
sched=torch.optim.lr_scheduler.ReduceLROnPlateau(opt,mode='min',patience=4,factor=0.5,verbose=True)

best=np.inf; patience=8; pat=0; best_state=None

def eval_loader(dl):
    model.eval(); losses=[]
    with torch.no_grad():
        for xb,yb in dl:
            xb,yb=xb.to(device), yb.to(device)
            losses.append(crit(model(xb), yb).item())
    return float(np.mean(losses))

for ep in range(1, 60+1):
    model.train(); tr=[]
    for xb,yb in train_loader:
        xb,yb=xb.to(device), yb.to(device)
        opt.zero_grad(); yhat=model(xb); loss=crit(yhat,yb)
        loss.backward(); torch.nn.utils.clip_grad_norm_(model.parameters(),1.0)
        opt.step(); tr.append(loss.item())
    val=eval_loader(val_loader); sched.step(val)
    print(f"Epoch {ep:03d} | train {np.mean(tr):.6f} | val {val:.6f}")
    if val < best - 1e-6:
        best=val; best_state={k:v.cpu().clone() for k,v in model.state_dict().items()}; pat=0
    else:
        pat+=1
        if pat>=patience:
            print("Early stopping"); break
if best_state is not None:
    model.load_state_dict(best_state); model.to(device)

### Evaluation in cfs (per step + overall

In [ ]:
def inverse_scale(Y_scaled):
    tmp=np.zeros((Y_scaled.size,1)); tmp[:,0]=Y_scaled.reshape(-1)
    inv=scaler.inverse_transform(tmp)[:,0]
    return inv.reshape(Y_scaled.shape)

def compute_metrics(y_true,y_pred):
    mae = np.mean(np.abs(y_true-y_pred),axis=0)
    rmse= np.sqrt(np.mean((y_true-y_pred)**2,axis=0))
    overall_mae=float(np.mean(np.abs(y_true-y_pred)))
    overall_rmse=float(np.sqrt(np.mean((y_true-y_pred)**2)))
    return mae, rmse, overall_mae, overall_rmse

model.eval(); preds=[]; truth=[]
with torch.no_grad():
    for xb,yb in test_loader:
        yhat=model(xb.to(device)).cpu().numpy()
        preds.append(yhat); truth.append(yb.numpy())
y_pred_s=np.vstack(preds); y_true_s=np.vstack(truth)
y_pred = inverse_scale(y_pred_s); y_true = inverse_scale(y_true_s)

mae_h, rmse_h, mae_all, rmse_all = compute_metrics(y_true, y_pred)
print("\n=== Test metrics (cfs) ===")
for i in range(H):
    print(f"+{(i+1)*10:>3} min: MAE={mae_h[i]:.4f} | RMSE={rmse_h[i]:.4f}")
print(f"Overall MAE:  {mae_all:.4f} cfs")
print(f"Overall RMSE: {rmse_all:.4f} cfs")


### Plotting

In [ ]:
import matplotlib.pyplot as plt
from datetime import timedelta

def plot_test_sample(sample_idx=0):
    """
    Plots one test window:
      - input history (last 24h = L points)
      - predicted next 6h (H points) vs observed next 6h
    """

    # --- pull raw series and timestamps from the test split ---
    series = test_df['Flow[cfs]']                      # raw (cfs), DatetimeIndex at 10-min
    idx = series.index

    # total number of test windows created by make_windows
    N_windows = len(series) - L - H + 1
    if N_windows <= 0:
        raise ValueError("Not enough test data to form a window. Check L/H and split sizes.")
    sample = max(0, min(sample_idx, N_windows-1))

    # time ranges for input and targets in the TEST split
    hist_slice = slice(sample, sample + L)                 # [i .. i+L-1]
    fut_slice  = slice(sample + L, sample + L + H)         # [i+L .. i+L+H-1]

    # inverse-scaled predictions for the whole test set were computed as y_pred (np.array [N,H])
    # We need the matching row for this sample:
    y_pred_sample = y_pred[sample]                         # shape [H]
    y_true_sample = y_true[sample]                         # shape [H]

    # gather timestamps
    hist_times = idx[hist_slice]
    fut_times  = idx[fut_slice]                            # these are t+10min..t+6h

    # values for the history (already raw cfs)
    hist_vals  = series.iloc[hist_slice].values

    # --- plot ---
    plt.figure(figsize=(12, 5))
    # history
    plt.plot(hist_times, hist_vals, label="History (last 24h)", linewidth=1.5)
    # future observed vs predicted
    plt.plot(fut_times, y_true_sample, label="Observed (next 6h)", linewidth=2)
    plt.plot(fut_times, y_pred_sample, label="Predicted (next 6h)", linestyle="--", linewidth=2)

    # visual cue at forecast boundary
    t_cut = hist_times[-1]
    plt.axvline(t_cut, linestyle=":", linewidth=1)
    plt.text(t_cut, max(np.nanmax(hist_vals), np.nanmax(y_true_sample))*1.01,
             " forecast start", rotation=90, va="bottom", ha="left")

    plt.title(f"Test sample #{sample} — 24h history + 6h forecast")
    plt.xlabel("Time")
    plt.ylabel("Flow (cfs)")
    plt.legend()
    plt.tight_layout()
    plt.show()

# Example: first sample
plot_test_sample(0)

# Or pick a middle/random sample
# plot_test_sample(100)


### Save

In [ ]:
torch.save(model.state_dict(),"lstm_flow_10min_h36_L144.pt")
joblib.dump(scaler,"scaler_minmax_flow.joblib")

### Inference helper (needs last 24h=144 points

In [ ]:
def forecast_next_6hrs(flow_last_24h_series):
    assert len(flow_last_24h_series)==L
    x=scaler.transform(flow_last_24h_series.values.reshape(-1,1))
    x_t=torch.from_numpy(x).float().unsqueeze(0).to(device)
    model.eval()
    with torch.no_grad():
        yhat_s=model(x_t).cpu().numpy()  # [1,36]
    return inverse_scale(yhat_s)[0]      # 36 forecasts in cfs

### Imports & config

In [ ]:
# ===================== 0) Imports & config =====================
import numpy as np, pandas as pd, random, math
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from torch import amp



In [ ]:
SEED=42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
H, L = 6, 144   # 6h horizon, 24h lookback (10-min steps)


### Data Prep

In [ ]:
df = flow_10min.copy().sort_index().asfreq('10min')
assert 'Flow[cfs]' in df.columns, "Expected a column named 'Flow[cfs]'"
df['Flow[cfs]'] = df['Flow[cfs]'].interpolate(limit_direction='both')



### 70/15/15 split

In [ ]:
N=len(df); n_tr=int(0.70*N); n_va=int(0.15*N)
train_df=df.iloc[:n_tr].copy()
val_df  =df.iloc[n_tr:n_tr+n_va].copy()
test_df =df.iloc[n_tr+n_va:].copy()


### daily baseline from TRAIN only

In [ ]:
def daily_baseline(train_series, index_like):
    tod = (index_like.hour*60 + index_like.minute)//10
    group = (train_series.index.hour*60 + train_series.index.minute)//10
    base = train_series.groupby(group).median()
    return base.reindex(tod).values

train_df['Baseline'] = daily_baseline(train_df['Flow[cfs]'], train_df.index)
val_df['Baseline']   = daily_baseline(train_df['Flow[cfs]'], val_df.index)
test_df['Baseline']  = daily_baseline(train_df['Flow[cfs]'], test_df.index)

# residual target
for d in (train_df, val_df, test_df):
    d['resid'] = d['Flow[cfs]'] - d['Baseline']

# time features
def add_time_feats(d):
    h = d.index.hour + d.index.minute/60.0
    d['sin_h'] = np.sin(2*np.pi*h/24.0).astype('float32')
    d['cos_h'] = np.cos(2*np.pi*h/24.0).astype('float32')
    dow = d.index.dayofweek
    d['sin_d'] = np.sin(2*np.pi*dow/7.0).astype('float32')
    d['cos_d'] = np.cos(2*np.pi*dow/7.0).astype('float32')
    return d
train_df=add_time_feats(train_df); val_df=add_time_feats(val_df); test_df=add_time_feats(test_df)

# previous-day residual feature
SHIFT_24H = 24*6
for d in (train_df, val_df, test_df):
    d['resid_prevday'] = d['resid'].shift(SHIFT_24H)

# drop first 24h in each split
for name, d in [('train',train_df),('val',val_df),('test',test_df)]:
    d.dropna(subset=['resid_prevday'], inplace=True)

# scale residuals (train only)
scaler = StandardScaler().fit(train_df[['resid']].values)
for d in (train_df, val_df, test_df):
    d['resid_z'] = scaler.transform(d[['resid']].values)

### arrays & sliding windows

In [ ]:
def base_matrix(d):
    X = d[['resid_z','resid_prevday','sin_h','cos_h','sin_d','cos_d']].astype('float32').values
    # return baseline (cfs) and flow (cfs) in case you need them later
    return X, d['Baseline'].values.astype('float32'), d['Flow[cfs]'].values.astype('float32')

def make_windows_mv(X, lookback=L, horizon=H):
    T=len(X); N=T - lookback - horizon + 1
    if N<=0:
        return np.empty((0,lookback,X.shape[1]),dtype='float32'), np.empty((0,horizon),dtype='float32')
    s0,s1 = X.strides
    Xw = np.lib.stride_tricks.as_strided(X, shape=(N,lookback,X.shape[1]),
                                         strides=(s0,s0,s1)).copy()
    # target: future residual_z (column 0)
    y  = np.lib.stride_tricks.as_strided(X[lookback:,0], shape=(N,horizon),
                                         strides=(s0,s0)).copy()
    return Xw, y

def future_time_feats(d, lookback=L, horizon=H):
    feats = d[['sin_h','cos_h','sin_d','cos_d']].astype('float32').values
    T=len(feats); N=T - lookback - horizon + 1
    if N<=0: return np.empty((0,horizon,4),dtype='float32')
    s0,s1 = feats.strides
    return np.lib.stride_tricks.as_strided(feats[lookback:], shape=(N,horizon,4),
                                           strides=(s0,s0,s1)).copy()

def future_baseline(B, lookback=L, horizon=H):
    B=B.astype('float32'); T=len(B); N=T - lookback - horizon + 1
    if N<=0: return np.empty((0,horizon),dtype='float32')
    s0=B.strides[0]
    return np.lib.stride_tricks.as_strided(B[lookback:], shape=(N,horizon),
                                           strides=(s0,s0)).copy()

# build arrays
Xtr, Btr, _ = base_matrix(train_df)
Xva, Bva, _ = base_matrix(val_df)
Xte, Bte, _ = base_matrix(test_df)

X_train, y_train = make_windows_mv(Xtr, L, H)
X_val,   y_val   = make_windows_mv(Xva, L, H)
X_test,  y_test  = make_windows_mv(Xte, L, H)

F_train = future_time_feats(train_df, L, H)
F_val   = future_time_feats(val_df,   L, H)
F_test  = future_time_feats(test_df,  L, H)

BL_train = future_baseline(Btr, L, H)
BL_val   = future_baseline(Bva, L, H)
BL_test  = future_baseline(Bte, L, H)

print("Shapes:",
      "\n  X_train", X_train.shape, "y_train", y_train.shape, "F_train", F_train.shape, "BL_train", BL_train.shape,
      "\n  X_val  ", X_val.shape,   "y_val  ", y_val.shape,   "F_val  ", F_val.shape,   "BL_val  ", BL_val.shape,
      "\n  X_test ", X_test.shape,  "y_test ", y_test.shape,  "F_test ", F_test.shape,  "BL_test ", BL_test.shape)


### Datasets / Loaders (Windows/Jupyter safe) 

In [ ]:
class TripletDS(Dataset):
    def __init__(self, X, y, F, BL):
        self.X=torch.from_numpy(X)
        self.y=torch.from_numpy(y)
        self.F=torch.from_numpy(F)
        self.BL=torch.from_numpy(BL)
    def __len__(self): return len(self.X)
    def __getitem__(self,i):
        return self.X[i].float(), self.y[i].float(), self.F[i].float(), self.BL[i].float()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_loader = DataLoader(TripletDS(X_train,y_train,F_train,BL_train),
                          batch_size=128, shuffle=True,  num_workers=0, pin_memory=(device.type=='cuda'))
val_loader   = DataLoader(TripletDS(X_val,y_val,F_val,BL_val),
                          batch_size=256, shuffle=False, num_workers=0, pin_memory=(device.type=='cuda'))
test_loader  = DataLoader(TripletDS(X_test,y_test,F_test,BL_test),
                          batch_size=256, shuffle=False, num_workers=0, pin_memory=(device.type=='cuda'))

### LSTM encoder–decoder with attention 

In [ ]:
class AttnSeq2Seq(nn.Module):
    def __init__(self, input_dim, enc_h=256, dec_h=256, horizon=H):
        super().__init__()
        self.horizon = horizon
        self.encoder = nn.LSTM(input_dim, enc_h, num_layers=2, batch_first=True)
        self.Wa = nn.Linear(enc_h, dec_h, bias=False)          # Luong-style projection
        self.dec_in = nn.Linear(1 + 4 + enc_h, dec_h)          # prev_y + 4 future time feats + context
        self.decoder = nn.LSTM(dec_h, dec_h, num_layers=1, batch_first=True)
        self.out = nn.Linear(dec_h, 1)

    def attend(self, dec_h_t, enc_out):
        # dec_h_t: [B, dec_h], enc_out: [B, L, enc_h]
        proj = self.Wa(enc_out)                                # [B, L, dec_h]
        scores = torch.einsum('bld,bd->bl', proj, dec_h_t)     # [B, L]
        alpha = torch.softmax(scores, dim=1)
        ctx = torch.einsum('bl,bld->bd', alpha, enc_out)       # [B, enc_h]
        return ctx

    def forward(self, x, future_feats, y_teacher=None, tf_ratio=0.5):
        B = x.size(0)
        enc_out, (h, c) = self.encoder(x)              # enc_out: [B, L, enc_h]
        dec_h = h[-1]                                  # [B, enc_h]
        dec_c = torch.zeros_like(dec_h).unsqueeze(0)   # [1, B, enc_h]
        dec_state = (dec_h.unsqueeze(0), dec_c)
    
        y_prev = torch.zeros(B, 1, device=x.device)    # [B, 1]
        outs = []
    
        for t in range(self.horizon):
            # attention context
            ctx = self.attend(dec_state[0].squeeze(0), enc_out)  # [B, enc_h]
    
            # ---- robust reshape to 2D before concat ----
            feat_t = future_feats[:, t, :] if future_feats.dim() == 3 else future_feats   # [B, 4]
            y_prev = y_prev.view(B, 1)                                                    # [B, 1]
            feat_t = feat_t.view(B, -1)                                                   # [B, 4]
            ctx    = ctx.view(B, -1)                                                      # [B, enc_h]
            din    = torch.cat([y_prev, feat_t, ctx], dim=1)                              # [B, 1+4+enc_h]
            # --------------------------------------------
    
            din = torch.relu(self.dec_in(din)).unsqueeze(1)  # [B, 1, dec_h]
            dec_out, dec_state = self.decoder(din, dec_state)
            y_t = self.out(dec_out).squeeze(1)               # [B]
    
            outs.append(y_t.unsqueeze(1))
            if self.training and (y_teacher is not None) and (np.random.rand() < tf_ratio):
                y_prev = y_teacher[:, t].unsqueeze(1)        # [B, 1]
            else:
                y_prev = y_t.detach().unsqueeze(1)           # [B, 1]
    
        return torch.cat(outs, dim=1)                         # [B, H]

input_dim = X_train.shape[2]
model = AttnSeq2Seq(input_dim=input_dim, enc_h=256, dec_h=256, horizon=H).to(device)

### Training (AMP-compatible across torch versions

In [ ]:
# ===================== 6) Training (fixed AMP + output shape) =====================
import torch
from torch import nn

# --- choose correct AMP API ---
if hasattr(torch, "amp") and hasattr(torch.amp, "GradScaler"):
    from torch import amp
    GradScaler = amp.GradScaler
    autocast   = amp.autocast
    autocast_kwargs = {"device_type": "cuda"} if torch.cuda.is_available() else {}
else:
    from torch.cuda.amp import GradScaler, autocast
    autocast_kwargs = {}

opt   = torch.optim.Adam(model.parameters(), lr=5e-3)
sched = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, mode='min', patience=4, factor=0.5)
crit  = nn.MSELoss()
scaler_amp = GradScaler(enabled=torch.cuda.is_available())

best = np.inf
patience = 10
pat = 0
best_state = None
train_losses, val_losses = [], []

# --- validation helper ---
def eval_loader(dl):
    model.eval()
    losses=[]
    with torch.no_grad():
        for xb,yb,Fb,BLb in dl:
            xb,yb,Fb = xb.to(device), yb.to(device), Fb.to(device)
            yhat = model(xb, Fb, None, 0.0)
            if yhat.ndim == 3 and yhat.shape[-1] == 1:
                yhat = yhat.squeeze(-1)  # <-- FIX
            loss = crit(yhat, yb)
            losses.append(loss.item())
    return float(np.mean(losses)) if len(losses)>0 else np.inf


# --- main train loop ---
for ep in range(1, 120+1):
    model.train()
    tr=[]
    for xb,yb,Fb,BLb in train_loader:
        xb,yb,Fb = xb.to(device), yb.to(device), Fb.to(device)
        opt.zero_grad(set_to_none=True)

        with autocast(**autocast_kwargs):
            yhat = model(xb, Fb, y_teacher=yb, tf_ratio=0.5)
            if yhat.ndim == 3 and yhat.shape[-1] == 1:
                yhat = yhat.squeeze(-1)  # <-- FIX
            loss = crit(yhat, yb)

        scaler_amp.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler_amp.step(opt)
        scaler_amp.update()
        tr.append(loss.item())

    # validation + scheduler
    val_loss = eval_loader(val_loader)
    sched.step(val_loss)

    train_losses.append(np.mean(tr))
    val_losses.append(val_loss)
    print(f"Epoch {ep:03d} | train {np.mean(tr):.6f} | val {val_loss:.6f}")

    if val_loss < best - 1e-6:
        best = val_loss
        best_state = {k:v.cpu().clone() for k,v in model.state_dict().items()}
        pat = 0
    else:
        pat += 1
        if pat >= patience:
            print("Early stopping")
            break

if best_state is not None:
    model.load_state_dict(best_state)
    model.to(device)

### evaluate in cfs (inverse residuals + add baseline)

In [ ]:
# ===================== Evaluation (robust shapes) =====================
import numpy as np

def _to_2d(a):
    # Accept [N,H] or [N,H,1] and return [N,H]
    return a.squeeze(-1) if (a.ndim == 3 and a.shape[-1] == 1) else a

def residz_to_flow(y_residz, BL):
    # y_residz, BL expected [N,H] (2-D)
    y_residz = _to_2d(y_residz)
    BL       = _to_2d(BL)
    resid = scaler.inverse_transform(y_residz.reshape(-1, 1)).reshape(y_residz.shape)
    return resid + BL

model.eval(); preds=[]; truth=[]; BLs=[]
with torch.no_grad():
    for xb, yb, Fb, BLb in test_loader:
        xb, Fb = xb.to(device), Fb.to(device)
        yhat = model(xb, Fb, None, 0.0).cpu().numpy()  # could be [B,H] or [B,H,1]
        preds.append(yhat)
        truth.append(yb.numpy())                        # [B,H]
        BLs.append(BLb.numpy())                         # [B,H]

y_pred_s = _to_2d(np.vstack(preds))  # [N,H]
y_true_s = _to_2d(np.vstack(truth))  # [N,H]
BL_arr   = _to_2d(np.vstack(BLs))    # [N,H]

y_pred = residz_to_flow(y_pred_s, BL_arr)  # cfs, [N,H]
y_true = residz_to_flow(y_true_s, BL_arr)  # cfs, [N,H]

def metrics(y_true, y_pred):
    mae  = np.mean(np.abs(y_true - y_pred), axis=0)                 # per-horizon
    rmse = np.sqrt(np.mean((y_true - y_pred)**2, axis=0))           # per-horizon
    mae_all  = float(np.mean(np.abs(y_true - y_pred)))              # overall
    rmse_all = float(np.sqrt(np.mean((y_true - y_pred)**2)))        # overall
    return mae, rmse, mae_all, rmse_all

mae_h, rmse_h, mae_all, rmse_all = metrics(y_true, y_pred)

print("\n=== Test metrics (cfs) ===")
for i in range(y_pred.shape[1]):
    print(f"+{(i+1)*10:>3} min: MAE={mae_h[i]:.4f} | RMSE={rmse_h[i]:.4f}")
print(f"Overall MAE: {mae_all:.4f} cfs | Overall RMSE: {rmse_all:.4f} cfs")


### plot a test sample 

In [ ]:
import matplotlib.pyplot as plt
test_series = test_df['Flow[cfs]']  # keep your original column name

def plot_test_sample(k=0):
    idx = test_series.index
    Nw = len(idx) - L - H + 1
    if Nw <= 0:
        raise ValueError("No test windows. Check L/H and split sizes.")
    k = max(0, min(k, Nw-1))
    hist = slice(k, k+L); fut = slice(k+L, k+L+H)
    ht, ft = idx[hist], idx[fut]
    hist_vals = test_series.iloc[hist].values
    plt.figure(figsize=(12,5))
    plt.plot(ht, hist_vals, label="History (24h)", lw=1.5)
    plt.plot(ft, y_true[k], label="Observed (6h)", lw=2)
    plt.plot(ft, y_pred[k], label="Predicted (6h)", lw=2, ls="--")
    t_cut = ht[-1]
    ymax = max(hist_vals.max(), y_true[k].max()) * 1.02
    plt.axvline(t_cut, ls=':', lw=1)
    plt.text(t_cut, ymax, "forecast start", rotation=90, va="bottom")
    plt.xlabel("Time"); plt.ylabel("Flow (cfs)")
    plt.title(f"Test sample #{k} — Feature-enhanced LSTM")
    plt.legend(); plt.tight_layout(); plt.show()

# Example:
plot_test_sample(0)

In [ ]:
def plot_test_sample(k=0):
    idx = test_series.index
    Nw = len(idx) - L - H + 1
    k = max(0, min(k, Nw-1))
    hist = slice(k, k+L)
    fut  = slice(k+L, k+L+H)
    ht, ft = idx[hist], idx[fut]
    hist_vals = test_series.iloc[hist].values

    # combine for a smooth line
    combined_time = np.concatenate([ht[-1:], ft])
    combined_obs  = np.concatenate([hist_vals[-1:], y_true[k]])

    plt.figure(figsize=(12,5))
    plt.plot(ht, hist_vals, label="History (24h)", lw=1.5)
    plt.plot(combined_time, combined_obs, label="Observed (6h)", lw=2)
    plt.plot(ft, y_pred[k], label="Predicted (6h)", lw=2, ls="--")

    t_cut = ht[-1]
    ymax = max(hist_vals.max(), y_true[k].max()) * 1.02
    plt.axvline(t_cut, ls=':', lw=1)
    plt.text(t_cut, ymax, "forecast start", rotation=90, va="bottom")
    plt.xlabel("Time"); plt.ylabel("Flow (cfs)")
    plt.title(f"Test sample #{k} — Feature-enhanced LSTM (1-hour horizon)")
    plt.legend(); plt.tight_layout(); plt.show()


In [ ]:
plot_test_sample(0)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

test_series = test_df['Flow[cfs]']  # keep your original column name

def plot_test_sample(k=0):
    idx = test_series.index
    Nw = len(idx) - L - H + 1
    if Nw <= 0:
        raise ValueError("No test windows. Check L/H and split sizes.")
    k = max(0, min(k, Nw-1))

    hist = slice(k, k+L)
    fut  = slice(k+L, k+L+H)
    ht, ft = idx[hist], idx[fut]
    hist_vals = test_series.iloc[hist].values

    # ---- build continuous curves ----
    obs_time = np.concatenate([ht[-1:], ft])          # join last history point
    obs_vals = np.concatenate([hist_vals[-1:], y_true[k]])
    pred_time = np.concatenate([ht[-1:], ft])
    pred_vals = np.concatenate([hist_vals[-1:], y_pred[k]])

    # ---- plot ----
    plt.figure(figsize=(12, 5))
    plt.plot(ht, hist_vals, label="History (24h)", lw=1.5, color='tab:blue')
    plt.plot(obs_time, obs_vals, label="Observed (1h)", lw=2, color='tab:orange')
    plt.plot(pred_time, pred_vals, label="Predicted (1h)", lw=2, ls="--", color='tab:green')

    t_cut = ht[-1]
    ymax = max(hist_vals.max(), y_true[k].max()) * 1.02
    plt.axvline(t_cut, ls=':', lw=1)
    plt.text(t_cut, ymax, "forecast start", rotation=90, va="bottom")
    plt.xlabel("Time"); plt.ylabel("Flow (cfs)")
    plt.title(f"Test sample #{k} — Feature-enhanced LSTM (1-hour horizon)")
    plt.legend(); plt.tight_layout(); plt.show()

# Example:
plot_test_sample(0)


## Attempt 3

In [ ]:
import numpy as np, pandas as pd, random
from math import sqrt
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler, StandardScaler # <-- ADD StandardScaler
import joblib

In [ ]:
SEED=42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
H, L = 18, 144   # 6h horizon, 24h lookback (10-min steps)

In [ ]:
df = flow_10min.copy().sort_index().asfreq('10min')
assert 'Flow[cfs]' in df.columns, "Expected a column named 'Flow[cfs]'"
df['Flow[cfs]'] = df['Flow[cfs]'].interpolate(limit_direction='both')


In [ ]:
N=len(df); n_tr=int(0.70*N); n_va=int(0.15*N)
train_df=df.iloc[:n_tr].copy()
val_df  =df.iloc[n_tr:n_tr+n_va].copy()
test_df =df.iloc[n_tr+n_va:].copy()

In [ ]:
def daily_baseline(train_series, index_like):
    tod = (index_like.hour*60 + index_like.minute)//10
    group = (train_series.index.hour*60 + train_series.index.minute)//10
    base = train_series.groupby(group).median()
    return base.reindex(tod).values

train_df['Baseline'] = daily_baseline(train_df['Flow[cfs]'], train_df.index)
val_df['Baseline']   = daily_baseline(train_df['Flow[cfs]'], val_df.index)
test_df['Baseline']  = daily_baseline(train_df['Flow[cfs]'], test_df.index)


################
# 1. Calculate the 'base' object as a Pandas Series
# (This is the logic from inside your function, but we stop before .values)
group = (train_df.index.hour * 60 + train_df.index.minute) // 10
base_series = train_df['Flow[cfs]'].groupby(group).median()

# 2. Save the Pandas Series to a CSV file
base_series.to_csv("daily_baseline_v2.csv", header=False)
print("Successfully saved daily_baseline.csv")
####################

# residual target
for d in (train_df, val_df, test_df):
    d['resid'] = d['Flow[cfs]'] - d['Baseline']

# time features
def add_time_feats(d):
    h = d.index.hour + d.index.minute/60.0
    d['sin_h'] = np.sin(2*np.pi*h/24.0).astype('float32')
    d['cos_h'] = np.cos(2*np.pi*h/24.0).astype('float32')
    dow = d.index.dayofweek
    d['sin_d'] = np.sin(2*np.pi*dow/7.0).astype('float32')
    d['cos_d'] = np.cos(2*np.pi*dow/7.0).astype('float32')
    return d
train_df=add_time_feats(train_df); val_df=add_time_feats(val_df); test_df=add_time_feats(test_df)

# previous-day residual feature
SHIFT_24H = 24*6
for d in (train_df, val_df, test_df):
    d['resid_prevday'] = d['resid'].shift(SHIFT_24H)

# drop first 24h in each split
for name, d in [('train',train_df),('val',val_df),('test',test_df)]:
    d.dropna(subset=['resid_prevday'], inplace=True)

# scale residuals (train only)
scaler = StandardScaler().fit(train_df[['resid']].values)
for d in (train_df, val_df, test_df):
    d['resid_z'] = scaler.transform(d[['resid']].values)

### Add residual derivative + (re)scale residuals & derivative

In [ ]:
from sklearn.preprocessing import StandardScaler
for d in (train_df, val_df, test_df):
    d['resid_diff'] = d['resid'].diff().fillna(0.0)

scaler = StandardScaler().fit(train_df[['resid','resid_diff']].values)

# --- ADD THIS LINE TO SAVE IT ---
# Use the *original* filename to overwrite the old file
joblib.dump(scaler, "flow_data_scaler.joblib")
print("✅ Successfully saved new flow_data_scaler.joblib")
# ---


for d in (train_df, val_df, test_df):
    d[['resid_z','resid_diff_z']] = scaler.transform(d[['resid','resid_diff']].values)


### Arrays & sliding windows (H=18, L=144)

In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
H, L = 18, 144   # 6-hour horizon, 24h lookback

def base_matrix(d):
    # NEW: include resid_diff_z (momentum)
    X = d[['resid_z','resid_diff_z','resid_prevday','sin_h','cos_h','sin_d','cos_d']].astype('float32').values
    return X, d['Baseline'].values.astype('float32'), d['Flow[cfs]'].values.astype('float32')

def make_windows_mv(X, lookback=L, horizon=H):
    T=len(X); N=T - lookback - horizon + 1
    if N<=0: return np.empty((0,lookback,X.shape[1]),dtype='float32'), np.empty((0,horizon),dtype='float32')
    s0,s1 = X.strides
    Xw = np.lib.stride_tricks.as_strided(X, shape=(N,lookback,X.shape[1]), strides=(s0,s0,s1)).copy()
    y  = np.lib.stride_tricks.as_strided(X[lookback:,0], shape=(N,horizon), strides=(s0,s0)).copy()
    return Xw, y

def future_time_feats(d, lookback=L, horizon=H):
    feats = d[['sin_h','cos_h','sin_d','cos_d']].astype('float32').values
    T=len(feats); N=T - lookback - horizon + 1
    if N<=0: return np.empty((0,horizon,4),dtype='float32')
    s0,s1 = feats.strides
    return np.lib.stride_tricks.as_strided(feats[lookback:], shape=(N,horizon,4), strides=(s0,s0,s1)).copy()

def future_baseline(B, lookback=L, horizon=H):
    B=B.astype('float32'); T=len(B); N=T - lookback - horizon + 1
    if N<=0: return np.empty((0,horizon),dtype='float32')
    s0=B.strides[0]
    return np.lib.stride_tricks.as_strided(B[lookback:], shape=(N,horizon), strides=(s0,s0)).copy()

Xtr,Btr,_ = base_matrix(train_df); Xva,Bva,_ = base_matrix(val_df); Xte,Bte,_ = base_matrix(test_df)
X_train,y_train = make_windows_mv(Xtr,L,H); X_val,y_val = make_windows_mv(Xva,L,H); X_test,y_test = make_windows_mv(Xte,L,H)
F_train = future_time_feats(train_df,L,H); F_val = future_time_feats(val_df,L,H); F_test = future_time_feats(test_df,L,H)
BL_train = future_baseline(Btr,L,H); BL_val = future_baseline(Bva,L,H); BL_test = future_baseline(Bte,L,H)

class TripletDS(Dataset):
    def __init__(self,X,y,F,BL): self.X=torch.from_numpy(X); self.y=torch.from_numpy(y); self.F=torch.from_numpy(F); self.BL=torch.from_numpy(BL)
    def __len__(self): return len(self.X)
    def __getitem__(self,i): return self.X[i].float(), self.y[i].float(), self.F[i].float(), self.BL[i].float()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_loader = DataLoader(TripletDS(X_train,y_train,F_train,BL_train), batch_size=128, shuffle=True,  num_workers=0, pin_memory=(device.type=='cuda'))
val_loader   = DataLoader(TripletDS(X_val,y_val,F_val,BL_val),       batch_size=256, shuffle=False, num_workers=0, pin_memory=(device.type=='cuda'))
test_loader  = DataLoader(TripletDS(X_test,y_test,F_test,BL_test),   batch_size=256, shuffle=False, num_workers=0, pin_memory=(device.type=='cuda'))

### Model: Attn Seq2Seq with bigger hidden & dropout

In [ ]:
class AttnSeq2Seq(nn.Module):
    def __init__(self, input_dim, enc_h=384, dec_h=384, horizon=H):
        super().__init__()
        self.horizon = horizon
        self.encoder = nn.LSTM(input_dim, enc_h, num_layers=2, batch_first=True, dropout=0.1)
        self.Wa = nn.Linear(enc_h, dec_h, bias=False)
        self.dec_in = nn.Linear(1 + 4 + enc_h, dec_h)   # prev_y + 4 future time feats + context
        self.decoder = nn.LSTM(dec_h, dec_h, num_layers=1, batch_first=True)
        self.out = nn.Linear(dec_h, 1)

    def attend(self, dec_h_t, enc_out):
        # enc_out: [B, L, enc_h], dec_h_t: [B, dec_h]
        proj = self.Wa(enc_out)                                # [B, L, dec_h]
        scores = torch.einsum('bld,bd->bl', proj, dec_h_t)     # [B, L]
        alpha = torch.softmax(scores, dim=1)                   # [B, L]
        ctx = torch.einsum('bl,bld->bd', alpha, enc_out)       # [B, enc_h]
        return ctx

    def forward(self, x, future_feats, y_teacher=None, tf_ratio=0.5):
        B = x.size(0)
        enc_out, (h, c) = self.encoder(x)                      # enc_out: [B, L, enc_h]
        dec_h = h[-1]                                          # [B, enc_h]
        dec_c = torch.zeros_like(dec_h).unsqueeze(0)           # [1, B, enc_h]
        dec_state = (dec_h.unsqueeze(0), dec_c)

        y_prev = torch.zeros(B, 1, device=x.device)            # [B, 1]
        outs = []

        for t in range(self.horizon):
            ctx = self.attend(dec_state[0].squeeze(0), enc_out)    # [B, enc_h]

            # ---- force everything to 2D before concat ----
            feat_t = future_feats[:, t]                           # expected [B, 4] but guard shapes:
            if feat_t.dim() == 3:  # e.g., [B,1,4]
                feat_t = feat_t.squeeze(1)
            feat_t = feat_t.reshape(B, -1)                        # [B, 4]
            y_prev = y_prev.reshape(B, 1)                         # [B, 1]
            ctx    = ctx.reshape(B, -1)                           # [B, enc_h]

            din = torch.cat([y_prev, feat_t, ctx], dim=1)         # [B, 1+4+enc_h]
            din = torch.relu(self.dec_in(din)).unsqueeze(1)       # [B, 1, dec_h]
            dec_out, dec_state = self.decoder(din, dec_state)     # [B, 1, dec_h]
            y_t = self.out(dec_out).squeeze(1)                    # [B]

            outs.append(y_t.unsqueeze(1))                         # [B, 1]
            # teacher forcing
            if self.training and (y_teacher is not None) and (np.random.rand() < tf_ratio):
                y_prev = y_teacher[:, t].unsqueeze(1)             # [B, 1]
            else:
                y_prev = y_t.detach().unsqueeze(1)                # [B, 1]

        return torch.cat(outs, dim=1)                             # [B, H]


In [ ]:
model = AttnSeq2Seq(input_dim=X_train.shape[2], enc_h=384, dec_h=384, horizon=H).to(device)


### Training (AMP-safe; LR=1e-3; output-shape fix)

In [ ]:
# ========= D) Training (AMP-safe; LR=1e-3; output-shape fix) =========
import torch
from torch import nn

# AMP compatibility for your torch version
if hasattr(torch, "amp") and hasattr(torch.amp, "GradScaler"):
    from torch import amp
    GradScaler = amp.GradScaler; autocast = amp.autocast
    autocast_kwargs = {"device_type":"cuda"} if torch.cuda.is_available() else {}
else:
    from torch.cuda.amp import GradScaler, autocast
    autocast_kwargs = {}

opt   = torch.optim.Adam(model.parameters(), lr=1e-3)  # LOWER LR
sched = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, mode='min', patience=4, factor=0.5)
crit  = nn.MSELoss()
scaler_amp = GradScaler(enabled=torch.cuda.is_available())

best = np.inf; patience = 15; pat = 0; best_state = None
train_losses, val_losses = [], []

def eval_loader(dl):
    model.eval(); losses=[]
    with torch.no_grad():
        for xb,yb,Fb,BLb in dl:
            xb,yb,Fb = xb.to(device), yb.to(device), Fb.to(device)
            yhat = model(xb, Fb, None, 0.0)
            if yhat.ndim == 3 and yhat.shape[-1] == 1:
                yhat = yhat.squeeze(-1)
            losses.append(crit(yhat, yb).item())
    return float(np.mean(losses)) if len(losses)>0 else np.inf

for ep in range(1, 150+1):
    model.train(); tr=[]
    for xb,yb,Fb,BLb in train_loader:
        xb,yb,Fb = xb.to(device), yb.to(device), Fb.to(device)
        opt.zero_grad(set_to_none=True)
        with autocast(**autocast_kwargs):
            yhat = model(xb, Fb, y_teacher=yb, tf_ratio=0.5)
            if yhat.ndim == 3 and yhat.shape[-1] == 1:
                yhat = yhat.squeeze(-1)    # make [B,H]
            loss = crit(yhat, yb)
        scaler_amp.scale(loss).backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler_amp.step(opt); scaler_amp.update()
        tr.append(loss.item())
    v = eval_loader(val_loader); sched.step(v)
    train_losses.append(np.mean(tr)); val_losses.append(v)
    print(f"Epoch {ep:03d} | train {np.mean(tr):.6f} | val {v:.6f}")
    if v < best - 1e-6:
        best = v; best_state = {k:v.cpu().clone() for k,v in model.state_dict().items()}; pat = 0
    else:
        pat += 1
        if pat >= patience:
            print("Early stopping"); break

if best_state is not None:
    model.load_state_dict(best_state); model.to(device)



In [ ]:
import torch

# Path where you want to save it
save_path = "lstm_flow_model.pth"

# Recommended: save both weights + metadata
torch.save({
    'model_state_dict': model.state_dict(),
    'input_dim': X_train.shape[2],
    'enc_h': 384,
    'dec_h': 384,
    'horizon': H,
    'scaler_mean': scaler.mean_,       # save scaling params too
    'scaler_scale': scaler.scale_,
    'L': L
}, save_path)

print(f"✅ Model saved to: {save_path}")

### Evaluation & plotting (unchanged, robust to shapes)

In [ ]:
# ========= E) Evaluation & plotting (robust shapes; correct inverse for 2-col scaler) =========
import numpy as np

def _to_2d(a):  # accept [N,H] or [N,H,1] -> [N,H]
    return a.squeeze(-1) if (a.ndim == 3 and a.shape[-1] == 1) else a

def residz_to_flow(y_residz, BL):
    y_residz = _to_2d(y_residz)   # [N,H]
    BL       = _to_2d(BL)         # [N,H]
    # scaler was fit on ['resid','resid_diff'] -> use only the first feature's stats
    resid = y_residz * scaler.scale_[0] + scaler.mean_[0]   # [N,H]
    return resid + BL                                       # flow (cfs), [N,H]

model.eval(); preds=[]; truth=[]; BLs=[]
with torch.no_grad():
    for xb,yb,Fb,BLb in test_loader:
        xb, Fb = xb.to(device), Fb.to(device)
        yhat = model(xb, Fb, None, 0.0).cpu().numpy()  # [B,H] or [B,H,1]
        preds.append(yhat)
        truth.append(yb.numpy())
        BLs.append(BLb.numpy())

y_pred_s = _to_2d(np.vstack(preds))   # [N,H]
y_true_s = _to_2d(np.vstack(truth))   # [N,H]
BL_arr   = _to_2d(np.vstack(BLs))     # [N,H]

y_pred = residz_to_flow(y_pred_s, BL_arr)  # cfs
y_true = residz_to_flow(y_true_s, BL_arr)  # cfs

def metrics(y_true, y_pred):
    mae  = np.mean(np.abs(y_true - y_pred), axis=0)
    rmse = np.sqrt(np.mean((y_true - y_pred)**2, axis=0))
    mae_all  = float(np.mean(np.abs(y_true - y_pred)))
    rmse_all = float(np.sqrt(np.mean((y_true - y_pred)**2)))
    return mae, rmse, mae_all, rmse_all

mae_h, rmse_h, mae_all, rmse_all = metrics(y_true, y_pred)
print("\n=== Test metrics (cfs) ===")
for i in range(H):
    print(f"+{(i+1)*10:>3} min: MAE={mae_h[i]:.4f} | RMSE={rmse_h[i]:.4f}")
print(f"Overall MAE: {mae_all:.4f} cfs | Overall RMSE: {rmse_all:.4f} cfs")


In [ ]:
# continuous plotting (option 3)
import matplotlib.pyplot as plt
test_series = test_df['Flow[cfs]']
def plot_test_sample(k=0):
    idx = test_series.index; Nw = len(idx) - L - H + 1; k = max(0, min(k, Nw-1))
    hist = slice(k, k+L); fut = slice(k+L, k+L+H)
    ht, ft = idx[hist], idx[fut]; hist_vals = test_series.iloc[hist].values
    obs_time  = np.concatenate([ht[-1:], ft]); obs_vals  = np.concatenate([hist_vals[-1:], y_true[k]])
    pred_time = np.concatenate([ht[-1:], ft]); pred_vals = np.concatenate([hist_vals[-1:], y_pred[k]])
    plt.figure(figsize=(12,5))
    plt.plot(ht, hist_vals, label="History (24h)", lw=1.5)
    plt.plot(obs_time,  obs_vals,  label="Observed (1h)", lw=2)
    plt.plot(pred_time, pred_vals, label="Predicted (1h)", lw=2, ls="--")
    t_cut = ht[-1]; ymax = max(hist_vals.max(), y_true[k].max()) * 1.02
    plt.axvline(t_cut, ls=':', lw=1); plt.text(t_cut, ymax, "forecast start", rotation=90, va="bottom")
    plt.xlabel("Time"); plt.ylabel("Flow (cfs)"); plt.title(f"Test sample #{k} — Feature-enhanced LSTM (A+B)")
    plt.legend(); plt.tight_layout(); plt.show()

# Example:
plot_test_sample(0)
